
# package学习
# jieba

## 安装jieba：

```
pip3 install jieba
```


```
!pip install jieba
```

[**python第三方库安装问题解决办法.pdf**](python第三方库安装问题解决办法.pdf)

## jieba常用函数

|jieba常用函数|作用|
|---|---|
|**jieba.lcut(text)**|将text进行分词，返回词语列表|
|jieba.cut(text)|与jieba.lcut()功能近似，主要是为了提高运行效率。返回的结构都是一个可迭代的generator|
|**jieba.load_userdict(file)**|为防止某些目标词被分割，需要载入自定义词典。|
|jieba.posseg(text)|标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。|


### 1.1. jieba.lcut(text)
将text进行分词，返回词语列表

In [1]:
import jieba

text = '我来到北京清华大学'
wordlist = jieba.lcut(text)
wordlist

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/y5/zfgjs5qx3kgcjxz7m4q4gltm0000gn/T/jieba.cache
Loading model cost 1.415 seconds.
Prefix dict has been built succesfully.


['我', '来到', '北京', '清华大学']

In [2]:
#保留词语字长度大于1的词
wordlist = [w 
            for w in wordlist 
            if len(w)>1]
wordlist

['来到', '北京', '清华大学']

### 1.2 jieba.cut(text)
返回的结构都是一个可迭代的generator, 能极大的提升运行效率。

generator可以通过for循环的方式提取出来。除非待分析文本量很大，一般都是用jieba.lcut()做文本分析。

In [3]:
import jieba

text2 = '我来到北京清华大学'
wordgenerator = jieba.cut(text2)
wordgenerator

<generator object Tokenizer.cut at 0x10606ede0>

In [4]:
for word in wordgenerator:
    print(word)

我
来到
北京
清华大学


### 1.3 jieba.load_userdict(file)	
导入自定义词典。为防止某些目标词被分割，需要载入自定义词典。

In [5]:
import jieba

text = '苦等6年！疯狂囤金震惊华尔街的中国大妈终于解套了'
wordlist = jieba.lcut(text)
wordlist

['苦',
 '等',
 '6',
 '年',
 '！',
 '疯狂',
 '囤金',
 '震惊',
 '华尔街',
 '的',
 '中国',
 '大妈',
 '终于',
 '解套',
 '了']

从上面的分词结果看， ``中国大妈``整体是一个词，为了防止``中国大妈``被分为``中国``和``大妈``，我们需要将``中国大妈``加到自定义词典中。

#### jieba.load_userdict(file)	

- file为自定义词典（txt文件）的路径。
- 词典格式：一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。
- 词典文件必须为 UTF-8 编码。

自定义词典格式，例如

```
创新办 3 i
云计算 5
凱特琳 nz
台中
中国大妈
```

In [1]:
import jieba

#导入自定义词典
jieba.load_userdict('data/diydict.txt')


text = '苦等6年！疯狂囤金震惊华尔街的中国大妈终于解套了'
wordlist = jieba.lcut(text)
wordlist

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/4m/fdp9_6v153z7j6q533c5cmn40000gp/T/jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built succesfully.


### 1.4 jieba.posseg(text)
标注句子分词后每个词的词性，采用和 中科院ictclas兼容的标记法。

In [2]:
import jieba.posseg as pseg

words = pseg.cut("我爱北京天安门")

#for part in words:
    #print(part)  
 
for w,f in words:
    print(w,f)

## 1.5 只保留中文
使用正则表达式剔除非中文字符，方便统计词频。中文汉字的正则表达式 **\u4e00-\u9fa5**

In [3]:
import re

test = 'dddd好的ddd好的good?*\~访问'
re.findall(r'[\u4e00-\u9fa5]', test)

In [6]:
test = 'dddd好的ddd好的good访问'
re.findall(r'[\u4e00-\u9fa5]+', test)

In [11]:
test = 'dddd好的!!!????ddd好LLL的good访问'
re.findall(r'[\u4e00-\u9fa5]+', test)

['好的', '好', '的', '访问']

In [16]:
test = 'dddd好的!!!????ddd好LLL的good访问'
words = re.findall(r'[\u4e00-\u9fa5]+', test)
#把列表转化成字符串
''.join(words)

'好的好的访问'

## 1.6 词频统计
使用jieba对《三体全集》做词频统计。**下面的程序运行有点慢，大概用了一分钟。**

In [32]:
import re
import jieba
import csv

# 读取文件中的文本
text = open('data/三体全集.txt').read()
#剔除非中文的内容（只保留中文）
text = ''.join(re.findall(r'[\u4e00-\u9fa5]+', text))

#jieba分词
wordlist = jieba.lcut(text)

wordset = [w for w in set(wordlist) if len(w)>1]

wordfreq = []
#词语计数
for word in wordset:
    freq = wordlist.count(word)
    wordfreq.append((word, freq))

    
# 词频排序
wordfreq = sorted(wordfreq, key=lambda k:k[1], reverse=True)


#存储词频分析结果
csvf = open('data/词频分析结果20192019.csv', 'a+', encoding='gbk', newline='')
writer = csv.writer(csvf)
writer.writerow(('word', 'freq'))

for word, freq in wordfreq:
    writer.writerow((word, freq))
    
csvf.close()

## 制作词云图
需要使用pyecharts库的WordCloud类

#### [pyecharts库官方文档](http://pyecharts.org/#/zh-cn/charts)

#### 安装方法:

在jupyter notebook中的安装，需要在cell中输入

```
!pip install pyecharts
```

或者在命令行中输入以下命令安装

```
pip install pyecharts
```

国人开发的数据可视化神库 pyecharts  https://dwz.cn/t5LZI9bP


In [1]:
from pyecharts.charts import Bar

bar = Bar()
bar.add_xaxis(["衬衫", "羊毛衫", "雪纺衫", "裤子"])
bar.add_yaxis("商家A", [5, 20, 36, 10])
# render 会生成本地 HTML 文件，默认会在当前目录生成 render.html 文件
# 也可以传入路径参数，如 bar.render("mycharts.html")
bar.render_notebook()

In [2]:
from pyecharts import options as opts
from pyecharts.charts import Page, WordCloud
from pyecharts.globals import SymbolType


words = [
    ("Sam S Club", 10000),
    ("Macys", 6181),
    ("Amy Schumer", 4386),
    ("Jurassic World", 4055),
    ("Charter Communications", 2467),
    ("Chick Fil A", 2244),
    ("Planet Fitness", 1868),
    ("Pitch Perfect", 1484),
    ("Express", 1112),
    ("Home", 865),
    ("Johnny Depp", 847),
    ("Lena Dunham", 582),
    ("Lewis Hamilton", 555),
    ("KXAN", 550),
    ("Mary Ellen Mark", 462),
    ("Farrah Abraham", 366),
    ("Rita Ora", 360),
    ("Serena Williams", 282),
    ("NCAA baseball tournament", 273),
    ("Point Break", 265),
]


wordcloud =WordCloud()

wordcloud.add("",
              words,
              word_size_range=[20,100])

wordcloud.set_global_opts(title_opts=opts.TitleOpts(title="基本示例"))


wordcloud.render_notebook()